# Writing tests

> How to write tests with `nbmodular`

In [ ]:
%load_ext nbmodular.core.cell2func

## Implement functions

In [ ]:
%%function data
a=1
b=2

In [ ]:
%%function add_function
c = a+b

## Writing tests with dependencies

It is best to define data variables in separate data functions, so that they can be retrieved and reused from different test functions:

In [ ]:
%%function first_data --test --data
a = 1

In [ ]:
%%function second_data --test --data
b = 2

Then we can define our test function. This function uses the variables defined in our previous data functions:

In [ ]:
%%function add_function --test
c = add_function (a, b)
assert c==3

In [ ]:
cell_processor = %cell_processor

In [ ]:
cell_processor.call_history

[('data', 'a=1\nb=2\n'),
 ('add_function', 'c = a+b\n'),
 ('first_data --test --data', 'a = 1\n'),
 ('second_data --test --data', 'b = 2\n'),
 ('add_function --test', 'c = add_function (a, b)\nassert c==3\n')]

In [ ]:
%print test_add_function --test

def test_add_function():
    a = test_first_data()
    b = test_second_data()
    b,a = data()
    c = add_function (a, b)
    assert c==3



In [ ]:
%print add_function

def add_function(b, a):
    c = a+b



If we try to define the same variable in another data function, we will get an error:

In [ ]:
%%function third_data --test --data
b = 3

ValueError: detected common variables with other test data functions {'b'}:

In [ ]:
%print_pipeline --test


from sklearn.utils import Bunch
from pathlib import Path
import joblib
import pandas as pd
import numpy as np

def test_test_tutorial_pipeline (test=True, prev_result=None, result_file_name="test_tutorial_pipeline"):
    result = test_tutorial_pipeline (test=test, load=True, save=True, result_file_name=result_file_name)
    if prev_result is None:
        prev_result = test_tutorial_pipeline (test=test, load=True, save=True, result_file_name=f"test_{result_file_name}")
    for k in prev_result:
        assert k in result
        if type(prev_result[k]) is pd.DataFrame:    
            pd.testing.assert_frame_equal (result[k], prev_result[k])
        elif type(prev_result[k]) is np.array:
            np.testing.assert_array_equal (result[k], prev_result[k])
        else:
            assert result[k]==prev_result[k]



In [ ]:
test_test_tutorial_pipeline 

<function __main__.test_test_tutorial_pipeline(test=True, prev_result=None, result_file_name='test_tutorial_pipeline')>

In [ ]:
test_test_tutorial_pipeline ()

In [ ]:
test_tutorial_pipeline ()

{}

In [ ]:
tutorial_pipeline 

NameError: name 'tutorial_pipeline' is not defined

In [ ]:
c = %cell_processor

In [ ]:
c.pipeline